# El problema de planificación de doblaje<br>
Nombre: Roberto Saul Cova Rocamora   <br>
Url: https://github.com/rscova/Algoritmos-de-Optimizacion-MIAR-2022

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las  tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de  doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de  grabación independientemente del número de tomas que se graben. No es posible grabar más  de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los  servicios de los actores de doblaje sea el menor posible.

Numero de tomas: 30 <br>
Numero de actores: 10 <br>

### Analisis de las posibles soluciones
El primer paso, es analizar el problema y las posibles soluciones que se van a generar para intuir que tipo de algoritmos serán más optimos para abordar este problema.

Si no se tiene en cuenta ninguna restricción es extremadamente complejo, ya que podríamos grabar desde todas en 1 día, hasta en 30 días, teniendo en cuenta que no hay límite de escenas por día. Por ejemplo, se podrían grabar 29 en 1 día, y hacer la última el último día. Y así con todas las posibles combianciones.

Si en vez de eso, tenemos en cuenta las restricciones de grabar 30 escenas en 5 días, con un máximo de 6 por día, hay que darse cuenta de que estamos ante un problema de combianción sin repetición.

El primer día podemos grabar 6 escenas de las 30 posibles, por tanto C(30,6). El segundo día, podemos elegir 30-6 escenas, por tanto C(24,6). Y así hasta llegar al día 5, que podremos elegir 6 escenas, por tanto C(6,6). Podriamos decir que es un factorial de combinaciones teniendo en cuenta de que vamos restando 6 cada vez. Entonces:
C(30,6) * C(24,6) * C(18,6) * C(12,6) * C(6,6) = 5.937.752.656

Pero esto no está completo, ya que si nos damos cuenta de que el orden de grabación por cada día no influye, por tanto, para cada día hay que dividirlo entre 6!, por tanto:
C(30,6) * C(24,6) * C(18,6) * C(12,6) * C(6,6) / (6!^5) = 75.287.520

Ahora, si tenemos en cuenta de que da igual en qué día en concreto se graban las escenas, habría que dividir el resultado entre 5!, por tanto:
C(30,6) * C(24,6) * C(18,6) * C(12,6) * C(6,6) / (6!^5 * 5!) = 49.481.272

Aunque hemos ido acontando el numero total de soluciones con las restriciones, el total de soluciones es demasiado grande para poder resolverlo por algoritmos de busqueda exhaustiva, como puede ser Vuelta atrás o fuerza bruta.



### Modelizacion del problema

Nos encontramos ante un problema de búsqueda de las soluciones para minimizar el coste de grabar todas las sesiones de doblaje.

Dadas las restricciones del problema, una buena aproximación para modelizar el espacio de soluciones sería implementar arrays de arrays. Si lo llevamos a código Python, serían lista de listas. En concreto una una lista (solución) de 5 listas (días) de 6 elementos cada una (tomas por día).

En este problema en particular, y para hacer más sencilla la selección y el cruce, he considerado que un individuo es completo cuando tiene su genoma y su aptitud por tanto, la estructura de datos que he utilizado es una tupla que contiene como primer elemento la aptitud (coste de esa solucion) y los cromosomas (la solución). Y la poblacion se obtiene como una lista ordenada segun el primer elemento de la tupla.


### Diseño del algoritmo

**Fuerza Bruta**

Diseñar un algoritmo que calcule por fuerza bruta todas las posibles combinaciones, es decir, realizar una busqueda exhaustiva, con 30 tomas de grabación restringiendo el problema a 5 días de grabación y 6 tomas por día tiene un coste exponencial, por tanto, para nuestro problema es inviable utilizar fuerza bruta.

Los pasos a seguir sería:
1. Generar todas las permutaciones posibles de las escenas y asignarlas a cada uno de los días, y luego asignar a cada actor las escenas correspondientes.
2. Calcular el coste total de cada una de las posibles soluciones
3. Quedarse con la de menor coste

La complejidad del algoritmo por fuerza bruta es factorial O(n!) donde n es el número de tomas totales, 30.

**Algoritmo Genético**

Los algoritmos genéticos son algoritmos de búsqueda que se inspiran en la evolución biológica y utilizan técnicas de selección, reproducción y mutación para explorar y explotar el espacio de búsqueda de soluciones.

Dado que tienen un buen balance entre intensifición y diversificación, pueden ayudar a encontrar una solución óptima o aproximada para este problema.

Para los datos de entrada, voy a utilizar una matriz de NumPy, donde las filas son las tomas, y las columnas son los actores (30,10). En cada pareja fila, colummna hay un 1 si el actor participa en la toma, y un 0 en caso contrario.

En una primera versión utilize dataframes de Pandas, pero he consegido una versión más eficiente con Numpy. Lo mismo ocurre con como se ordena la poblacion. En una primera versión utilicé colas de priorirdad de la librería queue de Python y para la versión final me he decantado por usar una lista ordenada, pues al final es más eficiente y más sencillo.

La complejidad de este algoritmo es O(m * nlog(n)) donde m es el numero de generaciones y n el tamaño de la poblacion. Hay que tener en cuenta que segun mi implementación, es en la funcion *reproduccion()* el cuello de botella y donde se da esta complejidad ya se generan 4 hijos por cada par de padres, y sabiendo que nlogn se obtiene al ordenar la lista de la poblacion, realmente en el peor caso tendríamos 5nlog(5n), dicha constante, hace que el tiempo de ejecución suba

In [1]:
import numpy as np
import random
from copy import deepcopy


In [2]:
problema = np.array([[1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
                    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],
                    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
                    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
                    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],
                    [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
                    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],
                    [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],
                    [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],
                    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
                    [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                    [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                    [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
                    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
                    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],
                    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],
                    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
                    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                    [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],
                    [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],
                    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],
                    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
                    [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
                    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]])

In [3]:
# Crear un individo aleatorio
def generar_individuo(problema):
    tomas = random.sample(range(30),30)
    cromosomas = [sorted(tomas[:6]),sorted(tomas[6:12]),sorted(tomas[12:18]),sorted(tomas[18:24]),sorted(tomas[24:30])]
    return (aptitud(cromosomas,problema),sorted(cromosomas))

# Devuelve True si un individuo es factible
def es_factible(individuo):
    if len(set(individuo[0]+individuo[1]+individuo[2]+individuo[3]+individuo[4])) != 30:
        return False
    if len(individuo) != 5:
        return False
    for cromosoma in individuo:
        if len(cromosoma) != 6:
            return False
    return True

# Calcula la aptitud de cada cromosoma
def aptitud(cromosomas,problema):
    coste = 0
    for cromosoma in cromosomas:
        coste += sum((np.sum(problema[cromosoma,:],axis=0,dtype=bool)))
    return coste

# Genera una poblacion aleatoria
def generar_poblacion(tam_poblacion,problema):
    poblacion = []
    for _ in range(tam_poblacion):
        poblacion.append(generar_individuo(problema))
    poblacion.sort(key=lambda x: x[0], reverse=False)
    return poblacion

In [4]:
poblacion = generar_poblacion(100,problema)
print(poblacion[0])

(35, [[0, 2, 4, 17, 20, 28], [1, 9, 19, 21, 23, 25], [3, 6, 11, 13, 14, 16], [5, 7, 8, 15, 24, 27], [10, 12, 18, 22, 26, 29]])


In [5]:
def seleccionar(poblacion,tam_poblacion,elitismo):
    n_elite = int(tam_poblacion*elitismo)

    # Cogemos los n mejores
    poblacion_elite = poblacion[:n_elite]

    # Cogemos los m por ruleta ordenados
    poblacion_random = random.sample(poblacion[n_elite:],(tam_poblacion-n_elite)) #sorted(random.sample(poblacion[n_elite:],(tam_poblacion-n_elite)))

    return (poblacion_elite+poblacion_random)

In [6]:
poblacion_elite = seleccionar(poblacion,10,0.7)

for ind in poblacion_elite:
    print(ind)

(35, [[0, 2, 4, 17, 20, 28], [1, 9, 19, 21, 23, 25], [3, 6, 11, 13, 14, 16], [5, 7, 8, 15, 24, 27], [10, 12, 18, 22, 26, 29]])
(35, [[0, 2, 3, 6, 26, 27], [1, 7, 13, 14, 18, 20], [4, 8, 10, 15, 24, 25], [5, 12, 16, 19, 21, 22], [9, 11, 17, 23, 28, 29]])
(36, [[0, 1, 5, 15, 21, 24], [2, 3, 4, 10, 17, 25], [6, 11, 13, 18, 27, 29], [7, 9, 12, 16, 20, 28], [8, 14, 19, 22, 23, 26]])
(36, [[0, 4, 10, 15, 18, 20], [1, 7, 8, 19, 21, 23], [2, 12, 16, 26, 28, 29], [3, 5, 6, 9, 14, 25], [11, 13, 17, 22, 24, 27]])
(36, [[0, 12, 13, 16, 17, 28], [1, 7, 10, 18, 27, 29], [2, 5, 14, 22, 25, 26], [3, 4, 9, 11, 19, 20], [6, 8, 15, 21, 23, 24]])
(36, [[0, 2, 3, 6, 11, 24], [1, 4, 9, 10, 12, 25], [5, 15, 16, 23, 26, 29], [7, 13, 17, 20, 22, 28], [8, 14, 18, 19, 21, 27]])
(36, [[0, 1, 8, 12, 22, 25], [2, 13, 14, 17, 21, 26], [3, 4, 7, 11, 27, 29], [5, 6, 9, 10, 19, 24], [15, 16, 18, 20, 23, 28]])
(37, [[0, 6, 8, 12, 24, 28], [1, 2, 7, 16, 19, 25], [3, 13, 17, 18, 20, 29], [4, 5, 10, 14, 15, 21], [9, 11, 22

In [7]:
def factibilizar(cromosoma):
    tomas = []
    # Obtenemos las tomas sin repetir
    [tomas.append(toma) for dia in cromosoma for toma in dia if toma not in tomas]
    # Obtenemos las tomas no repetidas y las mezclamos
    no_tomas = list(set(range(30))-set(tomas))
    random.shuffle(no_tomas)
    # Unimos las listas
    tomas = tomas + no_tomas
    # Creamos un cromosoma valido
    return [tomas[:6],tomas[6:12],tomas[12:18],tomas[18:24],tomas[24:30]]

def mutar(cromosoma,mutacion):
    # Intercambia los valores
    if random.uniform(0,1) <= mutacion:
        # Obtiene los indices de mutacion aleatorios
        i,j = random.sample(range(5),2)
        k,l = random.sample(range(6),2)
        cromosoma[i][k],cromosoma[j][l] = cromosoma[j][l],cromosoma[i][k]

    # Ordenar para obtener el cromosoma ordenado y poder compararlo
    for i in range(len(cromosoma)):
        cromosoma[i].sort()
    cromosoma.sort()
    
    return cromosoma

def generar_descendientes(padres,mutacion,problema):
    cromosomas = []
    descendencia = []
    p_corte = random.randint(1,4)
    # Genera 4 hijos usando un punto de corte aleatorio para mezclar los genes de los padres
    cromosomas.append(padres[0][1][:p_corte] + padres[1][1][p_corte:])
    cromosomas.append(padres[1][1][:p_corte] + padres[0][1][p_corte:])
    cromosomas.append(padres[0][1][p_corte:] + padres[1][1][:p_corte])
    cromosomas.append(padres[1][1][p_corte:] + padres[0][1][:p_corte])

    for i in range(4):
        cromosoma = factibilizar(cromosomas[i])
        cromosoma = mutar(cromosoma,mutacion)
        descendencia.append((aptitud(cromosoma,problema),cromosoma))
    return descendencia


def reproduccion(poblacion,mutacion,problema):
    poblacion_completa = deepcopy(poblacion)
    tam_poblacion = len(poblacion)
    # Genera los hijos a través de reproduccion aleatoria
    for i in range(tam_poblacion):
        padres = random.sample(poblacion,2)
        descendencia = generar_descendientes(padres,mutacion,problema)

        # Añade los hijos a la poblacion y evita soluciones repetidas
        for descendiente in descendencia:
            if descendiente not in poblacion_completa:
                poblacion_completa.append(descendiente)
    

    poblacion_completa.sort(key=lambda x: x[0], reverse=False)

    return poblacion_completa

In [8]:
poblacion_completa = reproduccion(poblacion_elite,0.4,problema)

for ind in poblacion_completa:
    print(ind)

print(len(poblacion_completa))

(35, [[0, 2, 4, 17, 20, 28], [1, 9, 19, 21, 23, 25], [3, 6, 11, 13, 14, 16], [5, 7, 8, 15, 24, 27], [10, 12, 18, 22, 26, 29]])
(35, [[0, 2, 3, 6, 26, 27], [1, 7, 13, 14, 18, 20], [4, 8, 10, 15, 24, 25], [5, 12, 16, 19, 21, 22], [9, 11, 17, 23, 28, 29]])
(35, [[0, 1, 5, 15, 21, 24], [2, 8, 12, 17, 20, 26], [3, 4, 6, 10, 18, 25], [7, 9, 11, 13, 27, 29], [14, 16, 19, 22, 23, 28]])
(36, [[0, 1, 5, 15, 21, 24], [2, 3, 4, 10, 17, 25], [6, 11, 13, 18, 27, 29], [7, 9, 12, 16, 20, 28], [8, 14, 19, 22, 23, 26]])
(36, [[0, 4, 10, 15, 18, 20], [1, 7, 8, 19, 21, 23], [2, 12, 16, 26, 28, 29], [3, 5, 6, 9, 14, 25], [11, 13, 17, 22, 24, 27]])
(36, [[0, 12, 13, 16, 17, 28], [1, 7, 10, 18, 27, 29], [2, 5, 14, 22, 25, 26], [3, 4, 9, 11, 19, 20], [6, 8, 15, 21, 23, 24]])
(36, [[0, 2, 3, 6, 11, 24], [1, 4, 9, 10, 12, 25], [5, 15, 16, 23, 26, 29], [7, 13, 17, 20, 22, 28], [8, 14, 18, 19, 21, 27]])
(36, [[0, 1, 8, 12, 22, 25], [2, 13, 14, 17, 21, 26], [3, 4, 7, 11, 27, 29], [5, 6, 9, 10, 19, 24], [15, 16, 18

In [41]:
def convergencia(poblacion):
    coste_medio = 0
    tam_poblacion = len(poblacion)
    for ind in poblacion:
        coste_medio += ind[0]
    return coste_medio/tam_poblacion
    

def algoritmo_genetico(problema,tam_poblacion=1_000,generaciones=100,elitismo=0.7,mutacion=0.8,generacion_print=1):
    print("Tamaño poblacion: ",tam_poblacion,"  Elitismo: ",elitismo, " Mutacion: ", mutacion)

    # Generamos la poblacion al azar
    poblacion = generar_poblacion(tam_poblacion,problema)

    for generacion in range(generaciones):
        # Selecion de la poblacion para quedarse con N individuos segun el elitimismo y ruleta
        poblacion = seleccionar(poblacion,tam_poblacion,elitismo)
        # Calculamos la convergencia de la poblacion
        coste_medio = convergencia(poblacion)
        # Cruzamos la poblacion
        poblacion = reproduccion(poblacion,mutacion,problema)
        
        if generacion % generacion_print == 0:
            print("Generacion #",generacion,"Convergencia de la generacion: ", coste_medio)
            print("  Mejor solucion: ", poblacion[0])
        
    print("Generacion #",generacion,"Convergencia de la generacion: ", coste_medio)
    print("  Mejor solucion: ", poblacion[0])
    return poblacion

Algo que me ha sido muy útil a la hora de analizar que sucede con la población a parte de saber el mejor individuo es saber como van evolucionando los demás individuos en general. Esto lo hago a través de la función *convergencia()* que me devuelve el coste medio de toda la poblacion una vez se ha realizado la selección. 

El siguiente código ejecuta la el algoritmo genético.

In [42]:
poblacion_final = algoritmo_genetico(problema,tam_poblacion=500,generaciones=100,elitismo=0.7,mutacion=0.8,generacion_print=10)

Tamaño poblacion:  500   Elitismo:  0.7  Mutacion:  0.8
Generacion # 0 Convergencia de la generacion:  38.176
  Mejor solucion:  (33, [[0, 1, 5, 8, 12, 16], [2, 3, 4, 7, 14, 20], [6, 15, 22, 24, 25, 27], [9, 17, 18, 19, 21, 29], [10, 11, 13, 23, 26, 28]])
Generacion # 10 Convergencia de la generacion:  32.086
  Mejor solucion:  (29, [[0, 1, 2, 5, 6, 12], [3, 4, 14, 15, 24, 27], [7, 8, 20, 26, 28, 29], [9, 10, 11, 19, 21, 25], [13, 16, 17, 18, 22, 23]])
Generacion # 20 Convergencia de la generacion:  30.658
  Mejor solucion:  (28, [[0, 1, 2, 3, 10, 14], [4, 7, 8, 20, 27, 28], [5, 9, 11, 19, 21, 25], [6, 12, 15, 24, 26, 29], [13, 16, 17, 18, 22, 23]])
Generacion # 30 Convergencia de la generacion:  29.978
  Mejor solucion:  (28, [[0, 1, 2, 3, 10, 14], [4, 7, 8, 20, 27, 28], [5, 9, 11, 19, 21, 25], [6, 12, 15, 24, 26, 29], [13, 16, 17, 18, 22, 23]])
Generacion # 40 Convergencia de la generacion:  29.718
  Mejor solucion:  (27, [[0, 1, 11, 19, 21, 29], [2, 3, 4, 12, 14, 26], [5, 6, 8, 15, 

### Posibles mejoras

**Multiarranque**

Una posible mejora para obtener mejores soluciones seria utilizar la técnica de multiarranque y guardar la mejor solución:

In [ ]:
print "Error no ejecutar"

mejor_individuo = (100,[])
n_arranques = 100
for i in range(n_arranques):
    print("##################################################################")
    print("Arranque #", i)
    individuo = algoritmo_genetico(problema,tam_poblacion=2000,generaciones=100,elitismo=0.7,mutacion=0.8)

    if individuo[0] < mejor_individuo[0]:
        mejor_individuo = individuo

print("##################################################################")
print("##################################################################")
print(mejor_individuo)

**Busqueda local**

In [34]:
def busqueda_local(poblacion,mejor_coste,problema):
    poblacion_completa = deepcopy(poblacion)
    for individuo in poblacion:
        cromosomas = deepcopy(individuo[1])
        for i in range(4):
            for j in range(6):
                for k in range(i+1,5):
                    for l in range(6):
                        cromosomas[i][j],cromosomas[k][l] = cromosomas[k][l],cromosomas[i][j]
                        apt = aptitud(cromosomas,problema)
                        if apt < mejor_coste:
                            print((apt,cromosomas))
                            poblacion_completa.append((apt,cromosomas))
                        cromosomas = deepcopy(individuo[1])

    return poblacion_completa

In [49]:
poblacion_busqueda_local = busqueda_local(poblacion_final,poblacion_final[0][0],problema)